In [1]:
# from google.colab import drive
# drive.mount('/content/drive',force_remount=True)


In [2]:
# %cd drive/My\ Drive/SIC\ -\ BigData/8.\ Project/

In [3]:
# %ls

## Import packages

In [4]:
# !pip install evaluate
# !pip install -U accelerate
!pip install -U datasets
# !pip install lxml
# !pip install -U pandas
# !pip install numpy
# !pip install transformers
# !pip install torch
# !pip install --upgrade pyarrow
# !pip install scikit-learn
# !pip install tensorboardX
# !pip install seaborn
# !pip install spacy
# !pip install xgboost
# !pip install beautifulsoup4

In [5]:
import re
import string
import glob
import sys, os
import string
import torch
import pandas as pd
import numpy as np
# import evaluate
import sklearn
import spacy
from bs4 import BeautifulSoup
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from datasets import DatasetDict, Dataset, load_dataset
from transformers import AutoTokenizer, TrainingArguments, Trainer, AutoModelForSequenceClassification, EarlyStoppingCallback
from spacy.lang.en import English
from spacy.lang.en.stop_words import STOP_WORDS
from datasets import Dataset, DatasetDict
stop_words = STOP_WORDS
punctuations = string.punctuation

import torch.nn as nn
from collections import Counter

In [6]:
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

## Prepare DataSet

In [7]:
model_name = "bert-base-uncased"
# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)


ds = load_dataset('takala/financial_phrasebank', 'sentences_50agree',trust_remote_code=True)
df = ds['train'].to_pandas()
# df = df.drop_duplicates(subset='sentence')
df.rename(columns={'sentence':'text'},inplace=True)

# 0 = negative
# 1 = neutral
# 2 = positive

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [8]:
df

,text,label
0,"According to Gran , the company has no plans t...",1
1,Technopolis plans to develop in stages an area...,1
2,The international electronic industry company ...,0
3,With the new production plant the company woul...,2
4,According to the company 's updated strategy f...,2
...,...,...
4841,LONDON MarketWatch -- Share prices ended lower...,0
4842,Rinkuskiai 's beer sales fell by 6.5 per cent ...,1
4843,Operating profit fell to EUR 35.4 mn from EUR ...,0
4844,Net sales of the Paper segment decreased to EU...,0


In [9]:
# Load dataset
temp_df, test_df = train_test_split(df, test_size=0.20, stratify=df['label'], random_state=42)
train_df, val_df = train_test_split(temp_df, test_size=0.20, stratify=temp_df['label'], random_state=42)

In [10]:
# Chuyển đổi ngược lại từ DataFrame sang Dataset
train_dataset = Dataset.from_pandas(train_df.reset_index(drop=True))
val_dataset = Dataset.from_pandas(val_df.reset_index(drop=True))
test_dataset = Dataset.from_pandas(test_df.reset_index(drop=True))

In [11]:
dataset = DatasetDict()
dataset['train'] = train_dataset
dataset['val'] = val_dataset
dataset ['test'] = test_dataset

In [12]:
# Truy cập vào tập dữ liệu train
train_dataset = dataset['train']

# Nếu nhãn được lưu trong cột 'label', thay 'label' bằng tên cột tương ứng
labels = train_dataset['label']

# Đếm số lượng các nhãn
label_count = Counter(labels)

In [13]:
print(label_count)

Counter({1: 1842, 2: 872, 0: 386})


## Preprocessing DataSet

#### A: Clean 'text' columns
- python -m spacy download en_core_web_sm

**Creating our tokenizer function - lemmatized, lowercased; remove "-PRON"s, stop_words, punctuations**

In [14]:
# nlp = English()
nlp = spacy.load('en_core_web_sm')
def clean_text(sentence):
    # # Parse HTML and extract plain text
    soup = BeautifulSoup(sentence+'<html>', "html.parser")
    sentence = re.sub(r'\[[^]]*\]', '', soup.get_text())

    # # # Remove links
    sentence = re.sub(r'http\S+|www\S+|https\S+', '', sentence, flags=re.MULTILINE)

    # # Remove words with invalid characters (keeping alphanumeric and %,'.')
    # sentence = ' '.join(word for word in sentence.split() if re.match(r'^[a-zA-Z0-9%.,:]+$', word))

    # # Process text with spaCy
    # doc = nlp(sentence)
    # mytokens = [word.lemma_.strip() if word.lemma_ != "-PRON-" else word for word in doc]

    # # Remove stop words and punctuation
    # mytokens = [word for word in mytokens if word not in stop_words and word not in punctuations]

    # Return cleaned text
    # return ' '.join(mytokens).strip()
    return sentence

In [15]:
print(clean_text('$QCOM chart: Qualcomm Short 61.8% Retracement, Trendline and Resistance. https://t.co/2WQqh35KOY'))

$QCOM chart: Qualcomm Short 61.8% Retracement, Trendline and Resistance. 


**Clean_text**

In [16]:
def preprocess_function(examples):
    examples['text'] = [clean_text(text) for text in examples['text']]
    return examples
cleaned_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/3100 [00:00<?, ? examples/s]

Map:   0%|          | 0/776 [00:00<?, ? examples/s]

Map:   0%|          | 0/970 [00:00<?, ? examples/s]

In [17]:
# Preprocess function
def tokenize(batch):
    return tokenizer(batch['text'], padding=True, truncation=True, max_length=64)

encoded_dataset = cleaned_dataset.map(tokenize, batched=True,batch_size=None)

Map:   0%|          | 0/3100 [00:00<?, ? examples/s]

Map:   0%|          | 0/776 [00:00<?, ? examples/s]

Map:   0%|          | 0/970 [00:00<?, ? examples/s]

In [18]:
encoded_dataset = encoded_dataset.remove_columns('text')

## Model

In [19]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [20]:
model = AutoModelForSequenceClassification.from_pretrained(model_name,num_labels=3).to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
class CustomClassifier(nn.Module):
    def __init__(self):
        super(CustomClassifier, self).__init__()

        self.fc1 = nn.Linear(768, 256)
        self.bn = nn.BatchNorm1d(256)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(256, 3)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.bn(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.softmax(x)

        return x

model.classifier = CustomClassifier()

In [22]:
print(model.classifier)

CustomClassifier(
  (fc1): Linear(in_features=768, out_features=256, bias=True)
  (bn): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
  (fc2): Linear(in_features=256, out_features=3, bias=True)
  (softmax): Softmax(dim=-1)
)


**Define Trainer parameters**

In [23]:
# import numpy as np

# def calculate_label_weights(label_counts):
#     total_samples = sum(label_counts)
#     label_weights = [np.sqrt(total_samples / count) for count in label_counts]
#     return label_weights

# # Giả sử bạn đã biết số lượng mẫu cho từng nhãn
# label_counts = [label_count[0], label_count[1], label_count[2]]
# label_weights = calculate_label_weights(label_counts)


In [24]:
# print(label_weights)

[2.833917444616472, 1.2972869041263797, 1.8854829279417071]


In [25]:
# label_weights = torch.tensor(label_weights, dtype=torch.float)

# print(label_weights)

# def compute_metrics(pred):
#     logits = pred.predictions
#     labels = pred.label_ids

#     # Chuyển đổi sang kiểu Float và Long tương ứng
#     logits = torch.tensor(logits, dtype=torch.float)
#     labels = torch.tensor(labels, dtype=torch.long)

#     # Tính toán xác suất dự đoán từ logits
#     probs = torch.softmax(logits, dim=-1)

#     # Tính cross entropy loss với trọng số
#     loss_fn = torch.nn.CrossEntropyLoss(weight=label_weights)
#     loss = loss_fn(probs, labels)

#     # Lấy nhãn dự đoán (nhãn với xác suất cao nhất)
#     preds = np.argmax(logits, axis=-1)

#     # Tính Macro F1 score
#     f1 = f1_score(labels.numpy(), preds, average='macro')

#     # Tính Accuracy
#     acc = accuracy_score(labels.numpy(), preds)

#     return {
#         "accuracy": acc,
#         "f1": f1
#     }


tensor([2.8339, 1.2973, 1.8855])


In [26]:
# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=10,
    learning_rate=1e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    eval_strategy='epoch',
    save_strategy="epoch",
    disable_tqdm = False,

    greater_is_better=False,
    load_best_model_at_end = True,
    metric_for_best_model='eval_loss',

    # fp16=True,
    # overwrite_output_dir=True,
    # seed=42,
    # local_rank=-1,
    # gradient_accumulation_steps=1,
    # warmup_steps=500,
)

# Compute metrics
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset['train'],
    eval_dataset=encoded_dataset['val'],
    compute_metrics=compute_metrics,
    # compute_metrics=compute_metrics_wrapper(label_weights),
    callbacks=[EarlyStoppingCallback(early_stopping_patience=1, early_stopping_threshold=.0)],
    optimizers= (torch.optim.AdamW(model.parameters(), lr=2e-5, eps=1e-5, betas=(0.9, 0.999)), None),
)

In [27]:
# Train the model
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.740430,0.827320,0.802674
2,0.822400,0.707647,0.841495,0.819530
3,0.679600,0.703917,0.851804,0.838959
4,0.638700,0.717380,0.836340,0.819362


TrainOutput(global_step=1552, training_loss=0.7103072048462543, metrics={'train_runtime': 268.2025, 'train_samples_per_second': 115.584, 'train_steps_per_second': 14.467, 'total_flos': 408758309222400.0, 'train_loss': 0.7103072048462543, 'epoch': 4.0})

In [28]:
trainer.evaluate()

{'eval_loss': 0.7039170265197754,
 'eval_accuracy': 0.8518041237113402,
 'eval_f1': 0.8389589407848824,
 'eval_runtime': 2.9796,
 'eval_samples_per_second': 260.434,
 'eval_steps_per_second': 32.554,
 'epoch': 4.0}

In [29]:
trainer.predict(encoded_dataset['test'])

PredictionOutput(predictions=array([[5.7344176e-03, 9.8353654e-01, 1.0729028e-02],
       [4.7713132e-03, 9.8611450e-01, 9.1141853e-03],
       [5.5106930e-03, 9.8522168e-01, 9.2676617e-03],
       ...,
       [4.7318990e-04, 3.3593268e-04, 9.9919087e-01],
       [5.3859933e-04, 3.3050004e-04, 9.9913090e-01],
       [6.5789875e-03, 9.8175669e-01, 1.1664397e-02]], dtype=float32), label_ids=array([1, 1, 1, 2, 0, 1, 2, 2, 0, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 2, 2, 1, 2,
       2, 2, 1, 0, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 2, 2, 0, 1, 1,
       1, 2, 2, 1, 1, 1, 0, 1, 2, 1, 0, 1, 1, 0, 2, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 2, 1, 2, 2, 1, 1, 0, 2, 0, 1, 1, 1, 2, 1, 0, 2, 1, 2, 1, 1,
       2, 1, 1, 0, 0, 2, 2, 1, 1, 1, 2, 0, 1, 2, 0, 1, 1, 1, 1, 1, 2, 1,
       1, 1, 0, 2, 1, 2, 2, 2, 2, 1, 1, 1, 2, 2, 0, 1, 0, 1, 2, 0, 1, 2,
       0, 1, 0, 2, 2, 2, 1, 2, 1, 1, 1, 2, 0, 1, 2, 2, 0, 2, 1, 2, 1, 2,
       0, 1, 2, 1, 1, 1,